In [6]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn.model_selection import train_test_split

from keras.callbacks import EarlyStopping

In [4]:
csv = pd.read_csv(
    "data.csv",
    delimiter=";"
)
df = csv[["Comment", "Spam"]]

df = df.dropna()


In [5]:

X = df['Comment'].values
y = df['Spam'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
    
print(len(X_train))
print(len(X_test))

model = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(model, input_shape=[], dtype=tf.string, trainable=True)

model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

print(model.summary())

1380
345


2023-04-09 19:56:34.359132: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-09 19:56:34.359775: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-09 19:56:34.377181: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-04-09 19:56:34.377647: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-04-09 19:56:34.377772: W tensorflow/c

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 50)                48190600  
                                                                 
 dense (Dense)               (None, 16)                816       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________
None


In [9]:


early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

# fit the model
model.fit(X_train,
         y_train,
         epochs=40,
         batch_size=512,
         validation_data=(X_test, y_test),
         verbose=1,
         callbacks=[early_stop]
)


model.save("trained-model.h5")

Epoch 1/40


2023-04-09 19:59:17.335601: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 192762400 exceeds 10% of free system memory.
2023-04-09 19:59:18.547123: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 192762400 exceeds 10% of free system memory.


1/3 [=========>....................] - ETA: 5s - loss: 0.7256 - accuracy: 0.3242

2023-04-09 19:59:19.946410: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 192762400 exceeds 10% of free system memory.
2023-04-09 19:59:20.054827: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 192762400 exceeds 10% of free system memory.


3/3 [==============================] - 4s 469ms/step - loss: 0.7119 - accuracy: 0.3804 - val_loss: 0.6995 - val_accuracy: 0.4783
Epoch 2/40
3/3 [==============================] - 1s 376ms/step - loss: 0.6946 - accuracy: 0.5101 - val_loss: 0.6866 - val_accuracy: 0.5652
Epoch 3/40
3/3 [==============================] - 1s 378ms/step - loss: 0.6829 - accuracy: 0.5775 - val_loss: 0.6763 - val_accuracy: 0.6319
Epoch 4/40
3/3 [==============================] - 1s 394ms/step - loss: 0.6730 - accuracy: 0.6529 - val_loss: 0.6669 - val_accuracy: 0.6696
Epoch 5/40
3/3 [==============================] - 1s 366ms/step - loss: 0.6637 - accuracy: 0.7051 - val_loss: 0.6581 - val_accuracy: 0.6899
Epoch 6/40
3/3 [==============================] - 1s 369ms/step - loss: 0.6545 - accuracy: 0.7384 - val_loss: 0.6493 - val_accuracy: 0.7188
Epoch 7/40
3/3 [==============================] - 1s 379ms/step - loss: 0.6453 - accuracy: 0.7536 - val_loss: 0.6406 - val_accuracy: 0.7565
Epoch 8/40
3/3 [===============

In [7]:
model_prod = tf.keras.models.load_model('./trained-model.h5', custom_objects={'KerasLayer':hub.KerasLayer})


2023-04-09 20:14:01.659233: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-09 20:14:02.200186: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 192762400 exceeds 10% of free system memory.


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
2023-04-09 20:14:07.453992: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 192762400 exceeds 10% of free system memory.
2023-04-09 20:14:07.566667: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 192762400 exceeds 10% of free system memory.
2023-04-09 20:14:11.543538: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 192762400 exceeds 10% of free system memory.


In [9]:
for test in ["Congratulations! You have been selected to receive a free gift worth $1000. Claim your prize by clicking this link and entering your personal information. Hurry, this offer is only available for a limited time!"]:
    out = model_prod.predict([test])
    print(out)

1/1 [==============================] - 0s 45ms/step
[[0.35363215]]
